# 1. Getting Connected

__Copy the following function, from saskia, into your bash_profile:__


    function fnal_proxy {


    echo "MAIN DQM http://localhost:3333 --> g2be2:3333"
    echo "TEST DQM http://localhost:3334 --> g2be2:4999"
    echo "MAIN MIDAS : http://localhost:8080 --> g2be1:8080"
    echo "MAIN OPERATIONS : http://localhost:8888 --> g2sc:80"
    echo " "
    echo "FTS http://localhost:8787 --> g2be3:8787/fts/status"
    echo " "
    echo "CCC : http://localhost:6003 --> g2web:7000"
    echo "CCC-JSON : http://localhost:6004 --> g2web:7001"
    echo "CALO SLOW CONTROL http://localhost:4999 --> g2sc:5000"
    echo " "
    echo "G2TRACKER0 MIDAS : http://localhost:8181 --> g2tracker0:8080"
    echo "GM2LIV DQM CLIENT : http://localhost:3434 --> gm2liv-priv:3344"
    echo "TRACKER WEB MONITOR : http://localhost:5001 --> g2tracker0-priv:5002"
    echo " "
    echo "EFFICIENCY/POT : http://localhost:6002 --> g2web:5002"
    echo " "
    echo "ONLINE PSQL DB: psql -U gm2_writer -d gm2_online_prod -h localhost -p 5434"
    echo ""
    echo "SRV : http://localhost:8989 --> gm2-01.dhcp.fnal.gov:80"
    echo "G2DB : http://localhost:9002 --> g2db:9002"

    ssh -nNT -L 9005:192.168.30.65:80 -L 6004:g2web:7001 -L 6003:g2web:7000 -L 6002:g2web:5002 -L 4999:g2sc:500 -L 3332:g2field-be:8080 -L 3331:g2field-be:3335 -L 3333:g2be2:3333 -L 3334:g2be2:4999 -L 8787:g2be3:8787 -L 8080:g2be1:8080 -o GSSAPIAuthentication=yes G2Muon@g2gateway01.fnal.gov -L 8888:g2sc:80 -L 3434:gm2liv-priv:3344 -L 8181:g2tracker0-priv:8080 -L 5001:g2tracker0-priv:5002 -L 8989:gm2-01.dhcp.fnal.gov:80 -L 5434:192.168.30.51:5433 -L 9002:g2db:9002 -L 9003:192.168.22.79:80 -L 9004:192.168.22.79:81

    }

__in a terminal, connect to the database with:__

    fnal_proxy

__run psql (if not already installed, do so using brew)__

    psql -U gm2_reader -d gm2_online_prod -h localhost -p 5434


__Within gm2_online_prod, to display ctags and time, do:__

    select * from gm2ctag_dqm limit 10;
    
__You can order by time, for example, by doing__

    select * from gm2ctag_dqm order by time ASC limit 10;

__to quit psql, run__

    \q

# 2. Reading ctags with python

__Run a demo script with python (the brew install on my home computer)__

    /usr/local/bin/python3.7 GetCtagsInTimeRange.py --db=localhost

# 3. Reading ctags with  Jupyter
__The following few lines of code shows how to__
* import a simple function to connect to the database to get muon info such as
    * time
    * ctags


In [ ]:
import sys
sys.path.append('/Users/bono/Desktop/gm2FieldAnalysis/MuonConvolution/muon_info')
from simple_ctags import get_ctags
import pandas as pd

In [ ]:
data = get_ctags("2018-04-22 00:00:00", "2018-04-25 00:00:00",'localhost')
df_muons = pd.DataFrame.from_dict(data, orient="index",columns=['ctags'])

df_muons.index.name = 'DateTime'
df_muons.index = pd.to_datetime(df_muons.index)
df_muons = df_muons.sort_values(by=['DateTime'])
# df_muons['EpochTime'] = df_muons.index.astype('int64')//1e9
print(df_muons.head())

In [ ]:
# import mpld3
# mpld3.enable_notebook()
# mpld3.disable_notebook()
df_muons.plot(figsize=(12,6))

# 4. Reading field info with Jupyter
__The following few lines of code shows how to__
* read in a text file from the field team to get
    * time
    * various multipole moments and their uncertainties

In [ ]:
path = "/Users/bono/Desktop/gm2FieldAnalysis/MuonConvolution/field_info/"
file = "60Hr_Average_Comparison_all_3956_3997_norescut.txt"
file = path + file
df_field = pd.read_csv(file, sep=" ", header=None)
df_field.columns = ["EpochTime", "D", "eD",
                    "NQ", "eNQ","SQ", "eSQ",
                    "NS", "eNS"]
df_field['DateTime'] = pd.to_datetime(df_field['EpochTime'],unit='s')
df_field = df_field.set_index('DateTime')
del df_field['EpochTime']
print(df_field.head())


In [ ]:
df_field.plot(y='D',figsize=(12,6))

# 5. A simple muon convolution

## 5a. The calculation

__Average field:__

We want the average B field experienced by the muons

\begin{equation}
\left\langle \textbf{B} \right\rangle = \textbf{B}(\textbf{x},T) \otimes M(\textbf{x},T)
\end{equation}

This invloves convolving the spacial and time structure of the muon beam and the B field. The spacial structures can be expressed as multipole expantions. The average expierienced B field can then be expressed as the sum of products of the moments

\begin{equation}
\left\langle \textbf{B} \right\rangle = \Sigma_i b_{i} \otimes m_{i}
\end{equation}

where the time structure has been made implicit. Call $t$ a time interval that is sufficiently small so that the spacial structure is effectivly constant. We can then factorize the convolution with

\begin{equation}
b_{i} \otimes m_{i} = \Sigma_t \frac{b_i(t)*m_i(t)}{\Sigma_t t} \equiv \Sigma_t \frac{p_i(t)}{\Sigma_t t} = \bar{p}_{i}
\end{equation}

Where we denote the time average of the quantity, $p$, as $\bar{p}$. We now can write 

\begin{equation}
\boxed{\left\langle \textbf{B} \right\rangle = \Sigma_{i} \bar{p}_{i}}
\end{equation}










__Error on average field:__

Assuming that the error in the muon and field structures are uncorrelated, and that the error in time measurements are zero, the error in the ith moment is

\begin{equation}
\Delta B_i \equiv \frac{dB_i}{db_i}\Delta b_i + \frac{dB_i}{dm_i}\Delta m_i = \frac{\Sigma_t m_i(t)}{\Sigma_t t} \Delta b_i + \frac{\Sigma_t b_i(t)}{\Sigma_t t} \Delta m_i
\end{equation}

or, more succinctly,

\begin{equation}
\boxed{\Delta B_i = \bar{m}_i \Delta b_i + \bar{b}_i \Delta m_i}
\end{equation}

To get the total error on the muon convoluted B field, we sum the errors

\begin{equation}
\boxed{\Delta \left\langle \textbf{B} \right\rangle 
    = \Sigma_{i} (\bar{m}_i \Delta b_i + \bar{b}_i \Delta m_i)}
\end{equation}

Finally, we seek to calculate
\begin{equation}
\boxed{ \left\langle \textbf{B} \right\rangle  +  (\Delta \left\langle \textbf{B} \right\rangle) 
    = \Sigma_{i}[ p_i + (\bar{m}_i \Delta b_i + \bar{b}_i \Delta m_i)} ]
\end{equation}




(Note that $m_i$ and $\Delta m_i$ come from the average beam position and RMS, as determined by the beam dynamics team, while $b_i$ and $\Delta b_i$ are the B-field multipole moments and their error, as determined by the field team)


__Calculation for a single moment__

For any given moment we have

\begin{equation}
B_i \pm (\Delta B_i) = \bar{p}_{i} \pm (\bar{m}_i \Delta b_i + \bar{b}_i \Delta m_i)
\end{equation}

Note that the ith muon moment, $m_i(t)$, is expressed as a dimensionless muliplicitive factor, representing spacial structure, times the muon populaton.

\begin{equation}
m_i(t) = s(t)*c(t) 
\end{equation}






__Calculation for the dipole moment__

The first moment is the dipole, which we will calculate by making the folling substutuions

\begin{equation}
b_1(t) \to D(t) 
\end{equation}

\begin{equation}
\Delta b_1(t) \to \Delta D(t)
\end{equation}
  
\begin{equation}
m_1(t) \to \text{c}(t)
\end{equation}

  
\begin{equation}
\Delta m_1(t) \to \Delta \text{c}(t)
\end{equation}


Explitly, we want to calculate the "averaged" dipole and its error

\begin{equation}
\boxed{ \left\langle \textbf{D} \right\rangle = 
\frac{\Sigma_t D_t * c_t}{\Sigma_t c_t} 
}
\end{equation}

asdf

\begin{equation}
\boxed{ 
\Delta \left\langle \textbf{D} \right\rangle = 
\frac{\Sigma_t c_t}{\Sigma_t t} \Delta D + 
\frac{\Sigma_t D_t}{\Sigma_t t} \Delta c
}
\end{equation}

## 5b. The approach 
__The overall goal here is to find the ctag weighted dipole field that the muons experience. This can be done by__
* synchronizing sampling rate and phase
* summing the product dipole of ctag in each time slice
* dividing that sum by the total number of ctags in all time slices

_We should take care to eliminate scaling artifacts that arise from resampling. For instance, cumulative quantities, such as the total number of muons, have the potential to be artificially scaled by the ratio of the resample and the original sample rate. We will therefore scale cumulative variables by this rato to undo the effect._

In [ ]:
#store the original sample rate for both dataframes
muons_time_interval = (df_muons.index.values[1].astype('int64')
                      - df_muons.index.values[0].astype('int64'))//1e9
field_time_interval = (df_field.index.values[1].astype('int64')
                      - df_field.index.values[0].astype('int64'))//1e9

#upsample each dataframe to 1 second                      
df_muons = df_muons.resample('1S').ffill()
df_field = df_field.resample('1S').ffill()

In [ ]:
df_total = df_field.join(df_muons)
df_total = df_total.fillna(0)
factor = 1/float(muons_time_interval)
df_total['ctags_per_second'] = df_total['ctags'].astype(float)*factor

In [ ]:
# mpld3.enable_notebook()
df_total.plot(y=['D','ctags'],figsize=(12,6))
df_total.plot(y='ctags_per_second',figsize=(12,6))

# df_total.plot(y='ctags')

__a simple convolution function__

In [ ]:
def convolution(start_time,stop_time,df,ctag_weighting_off=False):
    '''
    This function gets the ctag-weighted (or flat) average of the dipole field
    It takes in a time interval and a dataframe
    For every row in the dataframe, two columns (ctag and D) are multiplied. each product is summed
    Then that sum is divided by the sum of weights. 
    This is the defenition of the Weighted Arithmetic Mean
    '''
    
    #format the input
    start_time = pd.to_datetime(start_time)
    stop_time = pd.to_datetime(stop_time)
    #select times within the specified range
    mask = ( (df.index >= start_time) & (df.index <= stop_time))
    #first get the flat average
    flat_avg = float((df[mask]['D']).mean())
    #compute the average field experience by the muons
    #and how many muons experienced that average field
    avg_muon_field = float((df[mask]['D']*df[mask]['ctags']).sum())
    total_ctags = float(df[mask]['ctags'].sum())
    if (total_ctags > 0):
        avg_muon_field = float(avg_muon_field)/float(total_ctags)
    else:
        avg_muon_field = 0
        
    if(ctag_weighting_off):
        avg_muon_field = flat_avg
    
    return avg_muon_field, total_ctags

__show the effect that ctag weighting has over simple averaging, over some interval__

In [ ]:
t1 = 'april 10 2018  6:30pm'
t2 = "april 30 2018  6:30pm and 50 seconds"
avg_field, total_ctags = convolution(t1,t2,df_total,True)
print(avg_field, total_ctags)
avg_field, total_ctags = convolution(t1,t2,df_total,False)
print(avg_field, total_ctags)

## 5c. Plotting the results of the convolution

__First we will generate a number of time intervals and plot the difference between flat field averaging and ctag weighted averaging__

In [ ]:
import numpy as np
#first generate a sequence of dates
t1='2018-04-22 00:00:00'
t2s = pd.date_range(
    start='2018-04-22 08:00:00',
    end='2018-04-25 00:00:00',
    freq='H')

ctags = np.empty(0)
f_avg = np.empty(0)
c_avg = np.empty(0)
for t2 in t2s:
#     c = (convolution(t1,t2,df_total)[0])
    c,ctag = (convolution(t1,t2,df_total))
    ctags = np.append(ctags,ctag)
    c_avg = np.append(c_avg,c)
    f = (convolution(t1,t2,df_total,True)[0])
    f_avg = np.append(f_avg,f)



In [ ]:
d = {'EndTime': t2s, 'FlatAvg': f_avg, 'WeightedAvg': c_avg, 'ctags':ctags}

df_avgs = pd.DataFrame(d)
df_avgs = df_avgs.set_index('EndTime')
df_avgs['Difference'] = df_avgs['FlatAvg'] - df_avgs['WeightedAvg']
df_avgs = df_avgs.reset_index()
df_avgs['EndTime_epoch'] = df_avgs['EndTime'].astype('int64')//1e9
df_avgs = df_avgs.set_index('EndTime')

In [ ]:
df_avgs.plot(x='EndTime_epoch',y='ctags',grid=True,figsize=(12,6),kind='scatter')
df_avgs.plot(y=['FlatAvg','WeightedAvg'],grid=True,figsize=(12,6))
df_avgs.plot(x='EndTime_epoch',y='Difference',grid=True,figsize=(12,6),kind='scatter')

# df_avgs.plot(x='EndTime_epoch',y='WeightedAvg',grid=True,figsize=(12,6),kind='scatter')

__Next we will generate a number of time constant intervals and plot the difference between flat field averaging and ctag weighted averaging in each interval__

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
t2s = pd.date_range(
    start='2018-04-22 08:00:00',
    end='2018-04-25 00:00:00',
    freq='40min')


N = t2s.size


ctags = np.empty(0)
new_t1s = np.empty(0)
new_t2s = np.empty(0)
f_avg = np.empty(0)
c_avg = np.empty(0)
for i in range(N-1):
    t1=t2s[i]
    t2=t2s[i+1]
    new_t1s = np.append(new_t1s,t1)
    new_t2s = np.append(new_t2s,t2)
    
#     print(t1,t2)
    c,ctag = (convolution(t1,t2,df_total))
    c_avg = np.append(c_avg,c)
    ctags = np.append(ctags,ctag)
    f = (convolution(t1,t2,df_total,True)[0])
    f_avg = np.append(f_avg,f)
    
    

d = {'StartTime': new_t1s, 'StopTime': new_t2s, 'Interval': new_t2s - new_t1s,
     'FlatAvg': f_avg, 'WeightedAvg': c_avg, 'ctags':ctags}
df_const_int_avgs = pd.DataFrame(d)
df_const_int_avgs = df_const_int_avgs.set_index('StartTime')
df_const_int_avgs['Difference'] = df_const_int_avgs['FlatAvg'] - df_const_int_avgs['WeightedAvg']

df_const_int_avgs['StopTime_epoch'] = df_const_int_avgs['StopTime'].astype('int64')//1e9
df_const_int_avgs = df_const_int_avgs.reset_index()
df_const_int_avgs['StartTime_epoch'] = df_const_int_avgs['StartTime'].astype('int64')//1e9
# df_const_int_avgs = df_const_int_avgs.set_index('StartTime_epoch')

In [ ]:
# mpld3.enable_notebook()
#plot the ctags
# df_const_int_avgs.plot(x='StopTime_epoch', y='ctags',grid=True,figsize=(12,6),kind='scatter')

#overlay flat and weighted averages
ax = df_const_int_avgs.plot(x='StopTime_epoch', y='FlatAvg',color='blue',grid=True,figsize=(12,6),kind='scatter',alpha=0.5)
df_avgs.plot(x='EndTime_epoch',y='WeightedAvg',grid=True,figsize=(12,6),ax=ax,kind='scatter',color='black')
df_avgs.plot(x='EndTime_epoch',y='FlatAvg',grid=True,figsize=(12,6),ax=ax,kind='scatter',color='brown')
df_const_int_avgs.plot(x='StopTime_epoch', y='WeightedAvg',color='red',grid=True,figsize=(12,6),kind='scatter',ax=ax,alpha=0.5)

ax.set_xlabel("horizontal label")
ax.set_ylabel("vertical label")
plt.show()

#plot the difference
# df_const_int_avgs.plot(x='StopTime_epoch', y='Difference',grid=True,figsize=(12,6),kind='scatter')


__Now a resampling test__

In [ ]:
df_const_int_avgs['mult'] = (df_const_int_avgs['WeightedAvg']*df_const_int_avgs['ctags'])
df_const_int_avgs['mult_sum'] = df_const_int_avgs['mult'].cumsum()/df_const_int_avgs['ctags'].cumsum()
# df_const_int_avgs


In [ ]:
#time avg in each interval
ax = df_const_int_avgs.plot(x='StopTime_epoch', y='FlatAvg',
                            grid=True,figsize=(12,6),kind='scatter',alpha=0.5,color='blue')
#muon weighted avg in each interval
df_const_int_avgs.plot(x='StopTime_epoch', y='WeightedAvg',
                       grid=True,figsize=(12,6),kind='scatter',ax=ax,alpha=0.5,color='red')


#accumulated muon weighted avg in each interval
df_const_int_avgs.plot(x='StopTime_epoch', y='mult_sum',
                       grid=True,figsize=(12,6),kind='scatter',ax=ax,color='green')

#also the accumulated muon weighted avg in each interval
df_avgs.plot(x='EndTime_epoch',y='WeightedAvg',
             grid=True,figsize=(12,6),ax=ax,kind='scatter',color='black')

#accumulated time average
df_avgs.plot(x='EndTime_epoch',y='FlatAvg',
             grid=True,figsize=(12,6),ax=ax,kind='scatter',color='brown')



ax.set_xlabel("epoch time")
ax.set_ylabel("ppm")
plt.show()
print(df_const_int_avgs['mult_sum'].tail())



In [ ]:
ax = df_const_int_avgs.plot(x='StopTime_epoch', y='Difference',
                            grid=True,figsize=(12,6),kind='scatter',alpha=0.5,color='blue')

ax.set_xlabel("epoch time")
ax.set_ylabel("difference (ppm)")
plt.show()


# 6. A more elegant way to do the same convolution

__here we assume df_total, which already has all the field and ctag info, is being used for the call__

In [ ]:
def convolve_better(start_time,stop_time,df):
    #format the input
    start_time = pd.to_datetime(start_time)
    stop_time = pd.to_datetime(stop_time)
    #select times within the specified range
    mask = ( (df.index >= start_time) & (df.index <= stop_time))
    df = df[mask]
    
    #get a series of *cumulative* ctags
    s_ctags = df['ctags'].cumsum()
    #get a series of *cumulative* products
    s_avg = (df['D']*df['ctags']).cumsum()
    #divide the *cumulative* products by the cumulative ctags, to get the cumulative ctag weighted average
    s_avg = s_avg/s_ctags
    #get the flat average (or the time average) of the field
    s_time_avg = df['D'].expanding().mean()
    
    #combine the series into a df and return it
    all_series = [df['ctags'],s_ctags,s_avg,s_time_avg]
    df_new = pd.concat(all_series, axis=1)
    df_new.columns = ['new_ctags','ctags','muon_weighted_field','time_averaged_field']
    return df_new
    

In [ ]:
t1 = 'april 10 2018  6:30pm'
t2 = "april 30 2018  6:30pm and 50 seconds"
df_new = convolve_better(t1,t2,df_total)

In [ ]:
# mpld3.disable_notebook()
df_new[['muon_weighted_field','time_averaged_field']].fillna(method='ffill').plot(figsize=(12,6))
print(df_new)


In [ ]:
# mpld3.disable_notebook()
(df_new['time_averaged_field'] - df_new['muon_weighted_field']).fillna(method='ffill').plot(figsize=(12,6))

## Note that you  can make charts with plotly without slowing down the notebook by:

### 1 using fig.show(config={'staticPlot':True}), with the usual plotly.express.scatter

In [ ]:
import plotly.express as px
# Make a fig with plotly express 
fig = px.scatter(df_new.reset_index(), 
                 x="DateTime", 
                 y="muon_weighted_field")
#note that add_scatter doesn’t have the same “data frame + column names”
# API as Plotly Express, so it's called differently. 
# Note that add_scatter does allow for mixing of dfs, however...
# Also note, in the plot below, that add_scatter introduces some inconvient plot formatting defaults
fig.add_scatter(x=df_new.index, y=df_new['time_averaged_field'])

#The config={'staticPlot':True} option helps keep things running faster
fig.show(config={'staticPlot':True})

### or 2, using plotly.graph_objects.Scattergl (instead of Scatter), which plots online using WebGL

In [ ]:
import plotly.graph_objects as go
data = (df_new['time_averaged_field'] - 
        df_new['muon_weighted_field']).values
fig = go.Figure(data=go.Scattergl(y=data, mode='markers'))
fig.show()


## Note that the above results can easily be resampled with 
> resample('some value')).mean()

or


> resample('some value')).sum()

etc, depending on the type of quantity



In [ ]:
data = ((df_new['time_averaged_field'] -
         df_new['muon_weighted_field']).resample('H')).mean().values
# data
fig = go.Figure(data=go.Scattergl(y=data, mode='markers'))
fig.show()

In [ ]:
df_new.head()

In [ ]:
df_new['diff'] =  df_new['time_averaged_field'] - df_new['muon_weighted_field']
data1 = df_new['diff'].resample('20min').mean()
data1_max = (data1.max())
data2 = df_new['new_ctags'].resample('20min').sum()
data2 = data1_max*(data2)/(data2.max())
data3 = df_new['ctags'].resample('20min').sum()
data3 = data1_max*(data3)/(data3.max())

In [ ]:
#just a throw-away line to declare fig
fig = px.scatter(df_new)
#The two traces
fig.add_scatter(x=df_new.index, y=data1,mode='markers', name='field diff' )
fig.add_scatter(x=df_new.index, y=data2,mode='markers', name='ctags')
fig.add_scatter(x=df_new.index, y=data3, mode='markers', name='total ctags')
#plot non-interactivly
fig.show(config={'staticPlot':True})

In [ ]:
fig.update_layout(
    title=go.layout.Title(
        text="",
        x=0
    ),
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text="Time",
            font=dict(
                family="Courier New, monospace",
                size=18,
                color="#7f7f7f"
            )
        )
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text="ppm",
            font=dict(
                family="Courier New, monospace",
                size=18,
                color="#7f7f7f"
            )
        )
    )
)
fig.show()